## Ingesting PDF

In [13]:
%pip install --q unstructured langchain langchain-community
%pip install --q "unstructured[all-docs]" ipywidgets tqdm

/Users/tonykipkemboi/YouTube/Coding Projects/ollama_pdf_rag/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/Users/tonykipkemboi/YouTube/Coding Projects/ollama_pdf_rag/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from IPython.display import display as Markdown
from tqdm.autonotebook import tqdm as notebook_tqdm

C:\Users\13686\AppData\Local\Temp\ipykernel_27500\3632673901.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm as notebook_tqdm


In [2]:
# 方案1：使用 langchain 的 PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader

local_path = "WEF_The_Global_Cooperation_Barometer_2024.pdf"

# Local PDF file uploads
if local_path:
    loader = PyPDFLoader(file_path=local_path)
    data = loader.load()
else:
    print("Upload a PDF file")

In [3]:
# Preview first page
Markdown(data[0].page_content)

'The Global Cooperation \nBarometer 2024\nINSIGHT REPORT\nJANUARY 2024\nIn collaboration with \nMcKinsey & Company'

## Vector Embeddings

In [12]:
!ollama list

NAME                       ID              SIZE      MODIFIED       
nomic-embed-text:latest    0a109f422b47    274 MB    11 seconds ago    
qwen2.5:latest             845dbda0ea48    4.7 GB    6 weeks ago       
starcoder:latest           847e5a7aa26f    1.8 GB    6 weeks ago       
qwen:latest                d53d04290064    2.3 GB    6 weeks ago       
llama3.2:latest            a80c4f17acd5    2.0 GB    6 weeks ago       
llama3.1:latest            42182419e950    4.7 GB    8 weeks ago       


In [18]:
# # Pull nomic-embed-text model from Ollama if you don't have it
# !ollama pull nomic-embed-text
# # List models again to confirm it's available
# !ollama list

In [11]:
!ollama pull nomic-embed-text

^C


pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠹ pulling mani

In [4]:
# 1. First clean up any existing ChromaDB installations
# %pip uninstall -y chromadb
# %pip uninstall -y protobuf

# # 2. Install specific versions known to work together
# %pip install -q protobuf==3.20.3
# %pip install -q chromadb==0.4.22  # Using a stable older version
# %pip install -q langchain-ollama

# 3. Set the environment variable
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [5]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [6]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [7]:
# 1. First clean up any existing ChromaDB installations
# %pip uninstall -y chromadb
# %pip uninstall -y protobuf

# # 2. Install specific versions known to work together
# %pip install -q protobuf==3.20.3
# %pip install -q chromadb==0.4.22  # Using a stable older version
# %pip install -q langchain-ollama

# 3. Set the environment variable
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# 4. Now reimport with the new versions
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

# 5. Try creating the vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)

## Retrieval

In [8]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [9]:
!ollama list

NAME                       ID              SIZE      MODIFIED      
nomic-embed-text:latest    0a109f422b47    274 MB    5 minutes ago    
qwen2.5:latest             845dbda0ea48    4.7 GB    6 weeks ago      
starcoder:latest           847e5a7aa26f    1.8 GB    6 weeks ago      
qwen:latest                d53d04290064    2.3 GB    6 weeks ago      
llama3.2:latest            a80c4f17acd5    2.0 GB    6 weeks ago      
llama3.1:latest            42182419e950    4.7 GB    8 weeks ago      


In [10]:
# LLM from Ollama
local_model = "llama3.2"
llm = ChatOllama(model=local_model)

In [11]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [12]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [13]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
answer = chain.invoke("这个pdf文件讲了什么?")
print(type(answer))
print(answer)


<class 'str'>


In [34]:
# Delete all collections in the db
vector_db.delete_collection()